In [ ]:
# Part of the code for Andreev bound states calculation was adapted from https://doi.org/10.48550/arXiv.1810.04588

In [ ]:
import ipyparallel as ipp
cluster = ipp.Client()

In [ ]:
v = cluster[:]
lview = cluster.load_balanced_view()
len(v)

In [ ]:
%%px --local


import os
try:
    import mkl
    mkl.set_num_threads(1)
except:
    pass

os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 

In [ ]:
%%px --local

import scipy.linalg as la
import kwant
import tinyarray
import numpy as np
import matplotlib.pyplot as plt
import types
from scipy.constants import physical_constants
from scipy.signal import argrelextrema
import tinyarray
from ipywidgets import interact
plt.rcParams.update({'font.size': 32})
import pickle

In [ ]:
import adaptive
adaptive.notebook_extension()
import pandas as pd

In [ ]:
%%px --local

val_hJ= 6.62607004e-34 #value in J

val_e = physical_constants['elementary charge'][0]
val_hbar = physical_constants['Planck constant over 2 pi in eV s'][0]
val_m0 = physical_constants['electron mass energy equivalent in MeV'][0]

Phi_0=2.067*1e-15 
c = physical_constants['speed of light in vacuum'][0]
val_m0 = val_m0 / (c*10**9)**2 * 10**6 # [eV * s**2 / nm**2]
m=0.014
val_hbarJ = physical_constants['Planck constant over 2 pi'][0]*1e18 # in nm
mi_b= physical_constants['Bohr magneton in eV/T'][0]


sigma_0 = tinyarray.array([[1,0],[0,1]])   
sigma_x = tinyarray.array([[0,1],[1,0]])
sigma_y = tinyarray.array([[0,-1.j],[1.j,0]])
sigma_z = tinyarray.array([[1,0],[0,-1]])


tau_0 = tinyarray.array([[1,0],[0,1]]) 
tau_x = tinyarray.array([[0, 1], [1, 0]])
tau_y = tinyarray.array([[0, -1j], [1j, 0]])
tau_z = tinyarray.array([[1, 0], [0, -1]])

trs_m = -1j*sigma_y

dx=10

In [ ]:
%%px --local



par = types.SimpleNamespace(L=200, #Lengthc nm 
                            W=1000, #Width nm
                            t=val_hbar**2/val_m0/m/dx**2/2,
                            V_value=  .1,
                            tip=50, # in nm
                            mi=40e-3, # chemical potential
                            B=15.5e-3,
                            E=0,
                            phi=0,
                            xt=0,
                            g=0,
                            Delta=1e-3,
                            alpha=50e-3, 
                            dphi=np.pi*1e-13,
                            Peierls=1)
par.yt=par.L/2


In [ ]:
%%px --local


def make_system(par):
    
    
    half_W=int(par.W/2)
    t=par.t
    mi=par.mi
    V_value = par.V_value
    xt=par.xt
    g=par.g
    B=par.B
    alpha=par.alpha
    tip=par.tip
    yt=par.yt
    
    
    def system(pos):
        x, y  = pos
        ret = False
        if (-half_W <= x <= half_W and 0 <= y <= par.L):
            ret = True
        return ret

    def onsite(site):
        (x, y) = site.pos
        
        return (4*t-mi)*sigma_0 +1/2*par.g*sigma_z*B*mi_b + V_value/(1+( (x-xt)**2 +(y-yt)**2 )/tip**2)*sigma_0
            
        
    
    
    #########################################################
    
    def hopping_x(site1, site2):
        x1,y1 = site1.pos
        x2,y2 = site2.pos
        
        return (-t)*sigma_0 - 1.j/2/dx*sigma_y*alpha
    
    def hopping_y(site1, site2):
        x1,y1 = site1.pos
        x2,y2 = site2.pos
    
        return ( (-t)*sigma_0 + 1.j/2/dx*sigma_x*alpha)*np.exp( 1.j*val_e/val_hbarJ*B*((x1+x2)/2)*(y2-y1))
      
            #  Peiers phase
   
    ###################### superconducting contact without magnetic field B
    
    
    def hopping_x_contact(site1, site2):
        
        return (-t)*sigma_0 - 1.j/2/dx*sigma_y*alpha
        
    def hopping_y_contact(site1, site2):
        
        return (-t)*sigma_0 + 1.j/2/dx*sigma_x*alpha
    
    def superconducting(pos):
    
        return  (4*t-mi)*sigma_0 
    
    #########################################################
    
   


    
    def lead_bottom_shape(pos):
        (x, y) = pos
        ret = False
        if(-half_W <= x <= half_W):
            ret=True
        return ret
  
    def lead_top_shape(pos):
        (x, y) = pos
        ret = False
        if(-half_W <= x <= half_W):
            ret=True
        return ret
    

    

    
   
    
    sys = kwant.Builder()         
    
    lat = kwant.lattice.square(dx, norbs = 2)  
    sys[lat.shape(system, (0,0))] = onsite 
    sys[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    sys[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y

    
    
    lead_bottom = kwant.Builder(kwant.TranslationalSymmetry((0,-dx)),time_reversal=trs_m )
    lead_bottom[lat.shape(lead_bottom_shape, (0, 0))] = superconducting
    lead_bottom[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x_contact
    lead_bottom[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y_contact
    
    lead_top = kwant.Builder(kwant.TranslationalSymmetry((0,dx)),time_reversal=trs_m )
    lead_top[lat.shape(lead_top_shape, (0, 0))] = superconducting
    lead_top[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x_contact
    lead_top[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y_contact
    
 
    sys.attach_lead(lead_bottom)
   
    sys.attach_lead(lead_top)


    
    

    return sys

In [ ]:
sys = make_system(par)
sysf=sys.finalized()
kwant.plot(sysf, fig_size=(15,10), show = False);
plt.xlabel("W [nm]")
plt.ylabel("L [nm]")
# plt.savefig("kwant_sys.pdf")

In [ ]:
%%px --local

def ABS(smatrix, par):
    s = smatrix
    N, M = [len(li.momenta) // 2 for li in s.lead_info]
    s = s.data
    r_a11 = 1j * np.eye(N)
    r_a12 = np.zeros((N, M))
    r_a21 = r_a12.T
    r_a22 = 1j * np.exp(- 1j * par.phi) * np.eye(M)
    r_a = np.bmat([[r_a11, r_a12], [r_a21, r_a22]])
    zeros = np.zeros(shape=(len(s), len(s)))
    matrix = np.bmat([[zeros, (s.T.conj()).dot(r_a.conj())],
                      [(s.T).dot(r_a), zeros]])
    eigVl, eigVc  = la.eig(matrix)

    eigVlinds = eigVl.argsort()
    eigVl = eigVl[eigVlinds]
    eigVc = eigVc[:,eigVlinds]
    
    values = []
    vectors_e = []
    vectors_h = []

    for ii in range(len(eigVl)):
        if eigVl[ii].real >= 0 and eigVl[ii].imag >= 0:
            values.append(eigVl[ii].imag)
            vectors_e.append(eigVc.T[ii][0:len(eigVl) // 2] )
            vectors_h.append(eigVc.T[ii][len(eigVl) // 2:] )
                             
    
    
    return np.array(values)*par.Delta, np.array(vectors_e), np.array(vectors_h)

In [ ]:
%%px --local
def Ic_max(par,B,xt):
    par.xt=xt
    par.B=B
    sys = make_system(par)
    sysf=sys.finalized()
    
    smatrix = kwant.smatrix(sysf, par.E )
    #######
    phimin= 0
    phimax= 2*np.pi
    N = 101
    dphi=phimax / N

    phi_range=np.linspace(phimin, phimax, N)
    #######
    ABStab= [] # Here I save all ABS energies for diffrent phase
    
    
    for phi in phi_range:
        par.phi = phi
        ABStab.append( sum(ABS(smatrix,par )[0] ) )
    I= -1*np.gradient(ABStab) #current is -1* derivative d ABS/ d phi
        
    


    return max(I)/dphi/par.Delta

In [ ]:
%%px --local

def calc_energy(par,p,b,g):
    par.phi=p
    par.B=b
    par.g=g
    sys = make_system(par)
    sysf=sys.finalized()

    
    smatrix = kwant.smatrix(sysf, par.E ) 
    val,_,_  = ABS(smatrix,par)
    val.sort()

    return val

In [ ]:
%%px --local
sc_flux_quantum = 2.067833848*1e-15
L = par.L*1e-9
W = par.W*1e-9
B_period = sc_flux_quantum/L/W

In [ ]:
#SGM map
learner = adaptive.Learner2D(lambda xB :Ic_max(par=par,B =  xB[1], xt=xB[0]), bounds=[(-par.W/2-100,par.W/2+100),(-50e-3,50e-3)])
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.002)


In [ ]:
runner.live_info()

In [ ]:
# overwrites data
pd.DataFrame(learner.plot().image.I.data ).to_pickle('Icmap.pkl')    #to save the dataframe, df to *.pkl

In [ ]:
import matplotlib.colors as colors
def truncate_colormap(cmapIn='jet', minval=0.0, maxval=1.0, n=100):
    '''truncate_colormap(cmapIn='jet', minval=0.0, maxval=1.0, n=100)'''    
    cmapIn = plt.get_cmap(cmapIn)

    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmapIn.name, a=minval, b=maxval),
        cmapIn(np.linspace(minval, maxval, n)))

    

    return new_cmap

cmap = truncate_colormap(cmapIn='twilight_shifted',minval=0, maxval=0.85)

In [ ]:

map_ = pd.read_pickle('Icmap.pkl')
fig, ax = plt.subplots(1, figsize=(20,10))
im = ax.imshow(map_, cmap = truncate_colormap(cmapIn='twilight_shifted',minval=0, maxval=0.85) , aspect='auto', \
          extent=[-600,600,-4.84,4.84])

ax.set_xlabel("$x_{tip}$  [nm]")
ax.set_ylabel("$\Phi$  [$\Phi_0$]")
cbar = plt.colorbar(im)
cbar.set_label('$I_c$ [$e\Delta/\hbar$]')

### to plot lines
for i in range(-4,5):
    x = [-600, 600]
    t = i
    y = [t,t]
    plt.plot(x, y, color="white", linewidth=1.1)
    
for i in [-500,500]:
    y = [-4.84, 4.84]
    t = i
    x = [t,t]
    plt.plot(x, y, '--w', linewidth=1.1)
###
    

plt.savefig('Critical_current_vs_flux_and_tip_ts.pdf')
plt.show()

In [ ]:
N=300
xtip_range=np.linspace(-600,600,N)
out1 = lview.map_async(lambda tip :Ic_max(par=par,B = 1.5* B_period, xt=tip) , xtip_range)
out1.wait_interactive()
data2_xtip = out1.get()




In [ ]:
# Overwrites data 
pd.DataFrame(data2_xtip ).to_pickle('crossec_phi1,5.pkl')

In [ ]:

N=300
a = 106.92822300568442
b2 = 9.708941462684864 #Ic_max(par=par,B =  3/2*B_period, xt=1e6)

xtip_range=np.linspace(-600,600,N)

data2_xtip =np.array( pd.read_pickle('crossec_phi1,5.pkl') - b2 )  




a =   427.7128920227276#Ic_vs_xtB2(par, B=1.5e-3 * B_period , xt =  1e6,tau=1)
b =   290.2232347475518#Ic_vs_xtB2(par, B=1.5e-3 * B_period , xt =  1e6,tau=0.9)
c =   155.98544188658326#Ic_vs_xtB2(par, B=1.5e-3 * B_period , xt =  1e6,tau=0.6)
data1 = pd.read_pickle('analytics_shift_new1.pkl')
data2 = pd.read_pickle('analytics_shift_new09.pkl')
data3 = pd.read_pickle('analytics_shift_new06.pkl')




fig, ax1 = plt.subplots(figsize=(20,12))


ax1.set_xlabel('$x_{tip}$ $[nm]$')
ax1.set_ylabel('$I_{c}$-$I_{c0}$ [$e\Delta/\hbar$]',color='r')
lns1=ax1.plot(xtip_range, data2_xtip,'r',label='numerics')
ax1.tick_params(axis='y',labelcolor='r',labelsize=32)
ax1.tick_params(axis='x',labelsize=32)


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis





ax2.set_ylabel('$I_{c}$-$I_{c0}$ [$J_0 nm$]')  
lns2=ax2.plot(data1['x'],data1['y']-a, "k",label ="phenomenological model, τ = 1")
lns3=ax2.plot(data2['x'],data2['y']-b, "g",label ="phenomenological model, τ = 0.9")
lns4=ax2.plot(data3['x'],data3['y']-c, "b",label ="phenomenological model, τ = 0.6")
ax2.tick_params(axis='y', labelsize=32)

fig.tight_layout()

# added these three lines
lns = lns1+lns2+lns3+lns4
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc='lower center')
# ax2.legend(lns,labs,loc='upper center', bbox_to_anchor=(0.4, 1),
#           ncol=2, fancybox=True)
plt.xlim(-600,600)
ax1.grid()
# plt.savefig('analytics_shift_with_numerics.pdf')



plt.show()

In [ ]:
%%px --local
def andreev_wf(eigvec, kwant_wf):
    """
    Returns Andreev wavefunctions using eigenvalues and eigenvectors from
    the bound-state eigenvalue problem.

    Parameters
    ----------
    eigvec : numpy array
        Eigenvectors from the Andreev bound-state condition.
    kwant_wf : kwant.solvers.common.WaveFunction object
        Wavefunctions of a normal scattering region connected
        with two normal leads.
    """
    w = np.vstack((kwant_wf(0), kwant_wf(1))) 
    and_wf = [np.dot(vec, w) for vec in eigvec]
    return and_wf

In [ ]:
def calc_I_per_state(smatrix,par):

    
    t1,_,_=ABS(smatrix,par ) 
    
    par.phi+=par.dphi
    t2,_,_=ABS(smatrix,par)
    par.phi-=par.dphi
        
    I=(-1)*(t2-t1)/par.dphi/par.Delta
    return I

In [ ]:
def int_curr(sysf, current): #sum of currents in half of the map
    a,b = kwant.plotter.interpolate_current(sysf, current, relwidth=None, abswidth=None,n=9)
    val=[]
    s=np.zeros(a.shape[0])
    temp=[]
    for i in a[:,:,1]: 
        temp.append(i) #saving vertical values of current for every x position
    s=np.array(temp)


    for i in s:
        val.append(i[int(len(i)/2)]) #saving values in the middle of a system

    return sum(val) 
    

In [ ]:
def plot_Ic(par,name):
    #store currents before normalization
    current_e_wn = []
    current_h_wn = []

    
    

    
    sys = make_system(par)
    sysf=sys.finalized()
    
    #calculate scattering matrix
    smatrix = kwant.smatrix(sysf, par.E )
    
    eigval ,eigvec_e, eigvec_h = ABS(smatrix,par) 
    

    s_current=calc_I_per_state(smatrix,par=par)

    kwant_wf = kwant.wave_function(sysf, energy=par.E )

    I_operator = kwant.operator.Current(sysf).bind()
    
    

    Andreev_bs_e = andreev_wf(eigvec_e, kwant_wf)

    ### TO CALCULATE THE HOLE COMPONENTS OF ABS I NEED HOLE SCATTERING WAVE FUNCTIONS
    ### I obtain them by time reversing electron wave-functions so: Psi_h = \sigma_y Psi_e^*
    kwf0 = kwant_wf(0)
    kwf1 = kwant_wf(1)

    r = len(kwf0[0,:])//2

    sigma_y = np.array([[0, -1.j], [1.j, 0]])

    #I create a matrix with sigma_y as diagonal blocks to applu to each pair (spin_up, spin_down) of points in the wavefunction
    sgma = np.kron(np.eye(r),1j*sigma_y)

    #Iterate through wavefunctions
    w = np.conjugate(np.vstack(([np.matmul(sgma,vec) for vec in kwf0], [np.matmul(sgma,vec) for vec in kwf1])))
    Andreev_bs_h = [np.dot(vec, w) for vec in eigvec_h]
    
    
    
    
    added_current_e=0 # with normalization
    added_current_h=0 # with normalization
    
    for index,a_bsi in enumerate(Andreev_bs_e):
        
        #Electron current is just probability density current of electrons 
        #to get electric current we need to multiply it by electron charge -|e|
        
        current_by_mode_e = - I_operator(a_bsi)
        
        
        #The hole electric current is the current of time-reversed ELECTRON with reversed charge
        #TRS is applied to the Kwant wavefunctions, then we invert the charge in current distribution
        #The electric current is this current times |e|
        
        
        current_by_mode_h = - I_operator(Andreev_bs_h[index])
        

        #
        
        
        current_integrated_e = int_curr(sysf,current_by_mode_e)
        #current_e_wn.append(current_integrated_e)
        
        current_integrated_h = int_curr(sysf,current_by_mode_h)
        #current_h_wn.append(current_integrated_h)
        
        
        #Normalize the total current to the value obtained from the spectrum
        
        #Let us normalize but do not swap charges here (normalize to probability density, not current)
        #So swap the sign in the current from ABS
        #And then swap the signs to get probability current in each integrated currents
        temp = s_current[index]/(current_integrated_h + current_integrated_e)
        current_by_mode_e = current_by_mode_e*temp
        current_by_mode_h = current_by_mode_h*temp
        
        
        
        #Check if correctly normalized
        
        #current_integrated_e = int_curr(sysf,current_by_mode_e)
        #print("Electron current after normalization:", current_integrated_e)
        
        #current_integrated_h = int_curr(sysf,current_by_mode_h)
        #print("Hole current after normalization:", current_integrated_h)
    
    
        #print("Electric current from both:", current_integrated_e + current_integrated_h)
        #print("Should be:", s_current[index])
        
        added_current_e += current_by_mode_e
        added_current_h += current_by_mode_h
        
        
        
        
    
    
    #kwant.plotter.current(sysf, added_current_e, relwidth = 0.02, fig_size=(13,8), colorbar = 1,show=False)
    #kwant.plotter.current(sysf, added_current_h, relwidth = 0.02, fig_size=(13,8), colorbar = 1,show=False)

    kwant.plotter.current(sysf, added_current_e+added_current_h, relwidth = 0.02, fig_size=(11,8), colorbar = 0,show=False)
    plt.ylabel('y [nm]')
    plt.xlabel('x [nm]')
    #plt.title("Total electric current ")
    plt.savefig(name)
    plt.show()

 # First row of fig8 
 ## Current maps

In [ ]:
par.B=B_period/2
par.phi= np.pi/2
par.xt=700
plot_Ic(par,'fig8a.pdf')

In [ ]:
par.B=B_period
par.phi= 0
par.xt=700
plot_Ic(par,'fig8d.pdf')

In [ ]:
par.B=1.5*B_period
par.phi= -np.pi/2
par.xt=800
plot_Ic(par,'fig8g.pdf')

 # Second row of fig8
 ## SGM. With phase that gives critical current, for each tip position

In [ ]:
%%px --local
def Ic_max_tip(par,B,xt,yt):
    par.xt=xt
    par.yt=yt
    par.B=B
    sys = make_system(par)
    sysf=sys.finalized()
    
    smatrix = kwant.smatrix(sysf, par.E )
    #######
    phimin= 0
    phimax= 2*np.pi
    N = 100
    dphi=phimax / N

    phi_range=np.linspace(phimin, phimax, N)
    #######
    ABStab= [] # Here I save all ABS energies for diffrent phase
    
    
    for phi in phi_range:
        par.phi = phi
        ABStab.append( sum(ABS(smatrix,par )[0] ) )
    I= -1*np.gradient(ABStab) #current is -1* derivative d ABS/ d phi
        
    


    return max(I)/dphi/par.Delta

In [ ]:
learner = adaptive.Learner2D(lambda xy :Ic_max_tip(par=par, B=B_period/2, xt =  xy[0],yt = xy[1]), bounds=[(-par.W/2-100,par.W/2+100),(0,par.L)])
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.01)
runner.live_info()

In [ ]:
# overwrites data
pd.DataFrame(learner.plot().image.I.data ).to_pickle('fig8c.pkl')    #to save the dataframe, df to *.pkl

In [ ]:
a= 27.464313710834734 # Ic_max_tip(par=par, B=B_period/2, xt =  1e6,yt = 1e6)

out = pd.read_pickle('fig8c.pkl')
fig, ax = plt.subplots(1, figsize=(15,5))
im = ax.imshow( out -a , cmap='bwr',vmin=-12,vmax=12, \
          extent=[-600,600,0,200])


ax.set_xlabel("$x_{tip}$  [nm]")
ax.set_ylabel("$y_{tip}$  [nm]")
cbar = plt.colorbar(im)
cbar.set_label('$I_c -I_{c0}$ [$e\Delta/\hbar$]')

x = [-500, 500]

y = [0,200]

plt.plot([-500,-500], y, color="k", linewidth=1.1)
plt.plot([500,500], y, color="k", linewidth=1.1)

plt.savefig("fig8c.pdf")

plt.show()

In [ ]:
learner1 = adaptive.Learner2D(lambda xy :Ic_max_tip(par=par, B=B_period, xt =  xy[0],yt = xy[1]), bounds=[(-par.W/2-100,par.W/2+100),(0,par.L)])
runner1 = adaptive.Runner(learner1, executor=cluster, goal=lambda l: l.loss() < 0.01)
runner1.live_info()

In [ ]:
# overwrites data
pd.DataFrame(learner1.plot().image.I.data ).to_pickle('fig8f.pkl')    #to save the dataframe, df to *.pkl

In [ ]:
b = 1.6378348150982698 # Ic_max_tip(par=par, B=B_period, xt =  1e6 ,yt = 1e6)

out = pd.read_pickle('fig8f.pkl')
fig, ax = plt.subplots(1, figsize=(15,5))
im = ax.imshow( out-b  , cmap='bwr',vmin=-7.5,vmax=7.5, \
          extent=[-600,600,0,200])


ax.set_xlabel("$x_{tip}$  [nm]")
ax.set_ylabel("$y_{tip}$  [nm]")
cbar = plt.colorbar(im)
cbar.set_label('$I_c -I_{c0}$ [$e\Delta/\hbar$]')

x = [-500, 500]

y = [0,200]

plt.plot([-500,-500], y, color="k", linewidth=1.1)
plt.plot([500,500], y, color="k", linewidth=1.1)



plt.savefig("fig8f_h.pdf")

plt.show()

In [ ]:
learner2 = adaptive.Learner2D(lambda xy :Ic_max_tip(par, B=B_period*1.5, xt =  xy[0],yt = xy[1]), bounds=[(-par.W/2-100,par.W/2+100),(-50,par.L+50)])
runner2 = adaptive.Runner(learner2, executor=cluster, goal=lambda l: l.loss() < 0.002)
runner2.live_info()

In [ ]:
runner2.live_info()

In [ ]:
# overwrites data
pd.DataFrame(learner2.plot().image.I.data ).to_pickle('fig8i.pkl')    #to save the dataframe, df to *.pkl

In [ ]:
c = 9.571129075858718 # Ic_max_tip(par=par, B=B_period*1.5, xt =  1e6,yt = 1e6)

out = pd.read_pickle('fig8i.pkl')
fig, ax = plt.subplots(1, figsize=(15,5))
im = ax.imshow( out - c , cmap='bwr', \
          extent=[-600,600,0,200])


ax.set_xlabel("$x_{tip}$  [nm]")
ax.set_ylabel("$y_{tip}$  [nm]")
cbar = plt.colorbar(im)
cbar.set_label('$I_c -I_{c0}$ [$e\Delta/\hbar$]')

x = [-500, 500]

y = [0,200]

plt.plot([-500,-500], y, color="k", linewidth=1.1)
plt.plot([500,500], y, color="k", linewidth=1.1)



plt.savefig("fig8i.pdf")

plt.show()